In [ ]:
####################################################################################################################
# Code is based on the following documentation:
#===================================================================================================================
# https://docs.python.org/3.11/library/os.html
# https://matplotlib.org/
# https://www.tensorflow.org/
# https://keras.io/
# https://pandas.pydata.org/
# https://pillow.readthedocs.io/en/stable/
# https://seaborn.pydata.org/index.html
# https://pyyaml.org/wiki/PyYAMLDocumentation
# https://tqdm.github.io/
# https://plotly.com/
# https://arrow.apache.org/docs/python/
####################################################################################################################

In [1]:
import pyarrow.parquet as pq
import pandas as pd
import plotly.graph_objects as go
import warnings

warnings.filterwarnings('ignore')

In [2]:
df_exp1 = pq.read_table('./metrics/model_1_history.parquet').to_pandas()
df_exp2 = pq.read_table('./metrics/model_2_history.parquet').to_pandas()

In [3]:
print(df_exp1.columns.levels[0])
print(df_exp2.columns.levels[0])

Index(['DenseNet121', 'DenseNet169', 'DenseNet201', 'EfficientNetB0',
       'EfficientNetV2B0', 'EfficientNetV2L', 'ResNet101', 'ResNet152',
       'ResNet50', 'VGG16', 'VGG19'],
      dtype='object')
Index(['DenseNet121', 'DenseNet169', 'DenseNet201', 'EfficientNetB0',
       'EfficientNetV2B0', 'EfficientNetV2L', 'ResNet101', 'ResNet152',
       'ResNet50', 'VGG16', 'VGG19'],
      dtype='object')


In [4]:
print(df_exp1.columns.levels[1])
print(df_exp2.columns.levels[1])

Index(['accuracy', 'loss', 'val_accuracy', 'val_loss'], dtype='object')
Index(['accuracy', 'loss', 'val_accuracy', 'val_loss'], dtype='object')


In [5]:
def accuracy_data(df):
    train_accuracy = {}
    for parent_column in df.columns.levels[0]:
        accuracy_column = df.loc[:, (parent_column, df.columns.levels[1][0])]
        train_accuracy[parent_column + f'_{df.columns.levels[1][0]}'] = accuracy_column

    accuracy_df = pd.DataFrame(train_accuracy)

    val_accuracy = {}
    for parent_column in df.columns.levels[0]:
        accuracy_column = df.loc[:, (parent_column, df.columns.levels[1][2])]
        val_accuracy[parent_column + f'_{df.columns.levels[1][2]}'] = accuracy_column

    val_accuracy_df = pd.DataFrame(val_accuracy)

    return accuracy_df, val_accuracy_df

In [6]:
def format_df(df):
    return df.style \
    .highlight_max(color='red') \
    .highlight_max(color='green', axis=None) \
    .highlight_max(color='navy', axis=1, subset=df.index[-1]) \
    .format("{:.3%}", na_rep="-")

In [7]:
def plot_accuracy(df, title, xlabel='Epoch', ylabel='Accuracy'):
    fig = go.Figure()
    for column in df.columns:
        fig.add_trace(go.Scatter(x=df.index, y=df[column], mode='lines+markers', name=column))
    fig.update_layout(title=title, xaxis_title=xlabel, yaxis_title=ylabel, yaxis=dict(tickformat='.3%'), width=1000, height=500)
    fig.show()

# Experiment 1

Get training accuracy of models in each epoch

## Training Accuracy

In [8]:
accuracy_df, _ = accuracy_data(df_exp1)

From the below table, all models of `DenseNet` performs poorly as compared to other models. `ResNet` models give best result in this experiments but 100% training accuracy shows overfitting of data (Note: In first experiment, we didn't do any data augmentation or regularization). Apart from `DenseNet`, training accuracy values of all models is greater than 99%. `ResNet101` and `ResNet152` perform the best.

In [9]:
# Color Legend:
# - Red: maximum value in the column regardless of the epoch
# - Green: maximum value in whole dataframe
# - Navy: maximum value in the last row i.e. the last epoch

print("Training Accuracy")

format_df(accuracy_df)

Training Accuracy


,DenseNet121_accuracy,DenseNet169_accuracy,DenseNet201_accuracy,EfficientNetB0_accuracy,EfficientNetV2B0_accuracy,EfficientNetV2L_accuracy,ResNet101_accuracy,ResNet152_accuracy,ResNet50_accuracy,VGG16_accuracy,VGG19_accuracy
0,45.274%,44.619%,50.681%,95.786%,97.487%,98.226%,92.438%,92.164%,92.227%,78.424%,76.745%
1,63.608%,61.400%,66.438%,98.923%,99.651%,99.683%,97.846%,97.582%,97.381%,91.900%,92.037%
2,68.634%,65.984%,71.201%,99.324%,99.820%,99.736%,98.828%,98.828%,98.458%,94.118%,94.561%
3,71.518%,68.117%,72.595%,99.620%,99.852%,99.810%,99.588%,99.556%,99.123%,95.617%,96.198%
4,73.059%,71.169%,74.686%,99.704%,99.905%,99.863%,99.831%,99.863%,99.588%,96.663%,96.874%
5,74.316%,71.929%,75.890%,99.842%,99.958%,99.958%,99.937%,99.926%,99.768%,97.476%,97.803%
6,75.890%,72.669%,76.629%,99.916%,99.979%,99.937%,99.989%,99.989%,99.958%,98.184%,98.448%
7,76.312%,74.285%,77.495%,99.916%,99.958%,99.979%,100.000%,99.979%,100.000%,98.511%,98.849%
8,76.291%,74.739%,78.572%,99.926%,99.968%,99.937%,100.000%,100.000%,99.989%,99.145%,99.271%
9,77.812%,76.080%,79.216%,99.937%,99.989%,99.979%,100.000%,100.000%,100.000%,99.430%,99.514%


In [10]:
plot_accuracy(accuracy_df, 'Training Accuracy')

## Validation Accuracy

In [11]:
_, val_accuracy_df = accuracy_data(df_exp1)

Same pattern is shown in validation accuracy values as far as `DenseNet` is concerned. But `EfficientNetV2L` performs best among all models.

In [12]:
# Color Legend:
# - Red: maximum value in the column regardless of the epoch
# - Green: maximum value in whole dataframe
# - Navy: maximum value in the last row i.e. the last epoch

print("Validation Accuracy")

format_df(val_accuracy_df)

Validation Accuracy


,DenseNet121_val_accuracy,DenseNet169_val_accuracy,DenseNet201_val_accuracy,EfficientNetB0_val_accuracy,EfficientNetV2B0_val_accuracy,EfficientNetV2L_val_accuracy,ResNet101_val_accuracy,ResNet152_val_accuracy,ResNet50_val_accuracy,VGG16_val_accuracy,VGG19_val_accuracy
0,57.580%,55.618%,58.752%,98.904%,99.490%,99.720%,96.382%,96.076%,95.643%,88.178%,89.325%
1,65.605%,62.395%,67.873%,99.108%,99.592%,99.720%,96.994%,96.841%,96.968%,91.287%,91.822%
2,67.567%,66.497%,67.338%,99.083%,99.592%,99.720%,96.866%,97.070%,97.019%,91.975%,93.045%
3,69.656%,67.567%,67.465%,99.236%,99.669%,99.669%,96.968%,97.197%,96.943%,92.331%,93.452%
4,72.127%,66.599%,71.083%,99.185%,99.643%,99.669%,97.096%,97.248%,97.197%,92.280%,93.936%
5,73.503%,70.089%,71.185%,99.236%,99.669%,99.669%,97.146%,97.274%,96.968%,92.561%,93.554%
6,72.076%,70.318%,71.236%,99.236%,99.694%,99.694%,97.096%,97.401%,97.146%,93.045%,93.962%
7,73.274%,68.866%,72.510%,99.287%,99.745%,99.669%,97.146%,97.580%,97.503%,92.815%,94.191%
8,74.879%,67.975%,73.350%,99.312%,99.669%,99.720%,97.197%,97.580%,97.427%,92.866%,94.395%
9,73.809%,71.185%,73.045%,99.261%,99.643%,99.720%,96.994%,97.452%,97.427%,93.019%,94.217%


In [13]:
plot_accuracy(val_accuracy_df, 'Training Accuracy')

# Experiment 2

Get training accuracy of models in each epoch

## Training Accuracy

In [14]:
accuracy_df, _ = accuracy_data(df_exp2)

From the below table, all models of `DenseNet` performs poorly as compared to other models. `EfficientNet` models give best result in this experiments and none of the training accuracy is 100% it means there is no significant overfitting of data and our model is more generalized. This is introduced by input data augmentation and `Dropout` layer to minimize overfitting. `EfficientNetV2L` performs the best for training.

In [15]:
# Color Legend:
# - Red: maximum value in the column regardless of the epoch
# - Green: maximum value in whole dataframe
# - Navy: maximum value in the last row i.e. the last epoch

print("Training Accuracy")

format_df(accuracy_df)

Training Accuracy


,DenseNet121_accuracy,DenseNet169_accuracy,DenseNet201_accuracy,EfficientNetB0_accuracy,EfficientNetV2B0_accuracy,EfficientNetV2L_accuracy,ResNet101_accuracy,ResNet152_accuracy,ResNet50_accuracy,VGG16_accuracy,VGG19_accuracy
0,26.719%,28.028%,33.372%,79.322%,85.849%,92.048%,72.542%,73.007%,72.098%,47.861%,46.837%
1,39.962%,41.282%,46.140%,88.478%,93.040%,96.261%,84.581%,83.969%,83.177%,69.004%,69.089%
2,43.120%,44.281%,49.519%,89.260%,93.135%,96.452%,85.616%,86.461%,85.152%,73.883%,72.521%
3,46.721%,45.644%,50.343%,90.200%,93.769%,96.937%,86.461%,86.535%,85.416%,75.393%,75.531%
4,45.591%,46.013%,51.167%,90.738%,94.118%,96.663%,86.662%,87.074%,86.419%,76.692%,76.185%
5,46.679%,46.848%,51.558%,90.949%,94.255%,97.402%,87.158%,87.908%,86.387%,76.618%,75.879%
6,47.323%,46.922%,52.170%,90.548%,94.350%,97.117%,87.538%,87.982%,87.095%,78.023%,76.978%
7,47.080%,46.953%,52.012%,90.918%,94.751%,96.980%,87.644%,88.098%,87.031%,77.506%,76.840%
8,47.471%,46.658%,51.463%,90.886%,94.403%,97.159%,87.295%,88.341%,87.074%,77.231%,76.566%
9,47.629%,46.267%,51.959%,91.256%,94.477%,97.096%,87.475%,88.077%,86.915%,77.484%,76.904%


In [16]:
plot_accuracy(accuracy_df, 'Training Accuracy')

## Validation Accuracy

In [17]:
_, val_accuracy_df = accuracy_data(df_exp2)

Same pattern is shown in validation accuracy values as far as `DenseNet` is concerned. But `EfficientNetV2L` performs best among all models. Note, here validation accuracies are more than that of training, this is due to `Dropout` layer as it is applied at the time of training only.

In [18]:
# Color Legend:
# - Red: maximum value in the column regardless of the epoch
# - Green: maximum value in whole dataframe
# - Navy: maximum value in the last row i.e. the last epoch

print("Validation Accuracy")

format_df(val_accuracy_df)

Validation Accuracy


,DenseNet121_val_accuracy,DenseNet169_val_accuracy,DenseNet201_val_accuracy,EfficientNetB0_val_accuracy,EfficientNetV2B0_val_accuracy,EfficientNetV2L_val_accuracy,ResNet101_val_accuracy,ResNet152_val_accuracy,ResNet50_val_accuracy,VGG16_val_accuracy,VGG19_val_accuracy
0,31.694%,34.471%,30.599%,97.529%,99.159%,99.694%,93.656%,94.013%,91.261%,84.535%,84.076%
1,29.554%,33.274%,26.013%,98.166%,99.210%,99.694%,94.293%,94.828%,92.892%,86.446%,86.981%
2,40.000%,30.828%,28.306%,98.242%,99.338%,99.694%,94.497%,94.777%,92.713%,88.994%,88.688%
3,40.306%,27.796%,25.885%,98.497%,99.414%,99.694%,94.395%,94.522%,92.408%,89.682%,88.433%
4,38.140%,34.879%,27.287%,98.599%,99.363%,99.669%,94.904%,95.414%,93.911%,89.503%,89.656%
5,39.669%,38.217%,30.140%,98.471%,99.465%,99.694%,95.465%,95.567%,93.096%,88.815%,88.280%
6,26.344%,33.936%,29.911%,98.471%,99.312%,99.669%,95.185%,95.363%,92.866%,89.146%,90.089%
7,37.019%,25.325%,30.140%,98.522%,99.414%,99.669%,94.854%,95.159%,93.274%,89.045%,90.650%
8,33.707%,34.854%,24.459%,98.471%,99.439%,99.669%,95.210%,95.414%,93.885%,88.790%,90.369%
9,32.535%,31.210%,26.344%,98.675%,99.389%,99.669%,94.955%,95.057%,93.146%,89.299%,90.140%


In [19]:
plot_accuracy(val_accuracy_df, 'Training Accuracy')